## Dataset : 

https://data.cityofchicago.org/Public-Safety/Crimes-2022/9hwr-2zxp/about_data

# Athena Integration
This notebook demonstrates how to upload data to S3, create an Athena table, and query it.

In [ ]:
# Upload file to S3 using Boto3

import boto3
import os
from botocore.exceptions import NoCredentialsError, ClientError

# Config
bucket_name = 'chicago-crime-data-kinjal'
file_path = 'processed_data.csv'  # or 'processed_data.parquet'
s3_key = 'raw/processed_data.csv'

# Check if file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Local file not found: {file_path}")

# Initialize S3 client (uses credentials from ~/.aws/credentials or environment)
s3 = boto3.client('s3')

# Upload
try:
    s3.upload_file(file_path, bucket_name, s3_key)
    print(f"✅ File successfully uploaded to s3://{bucket_name}/{s3_key}")
except FileNotFoundError:
    print("The file was not found.")
except NoCredentialsError:
    print("AWS credentials not available.")
except ClientError as e:
    print(f"Unexpected error: {e}")


In [ ]:
# Athena Table Creation SQL
'''
CREATE EXTERNAL TABLE IF NOT EXISTS chicago_crime (
    id STRING,
    case_number STRING,
    date STRING,
    primary_type STRING,
    location_description STRING,
    arrest BOOLEAN,
    domestic BOOLEAN,
    beat INT,
    district INT,
    ward INT,
    community_area INT,
    year INT,
    latitude DOUBLE,
    longitude DOUBLE
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
LOCATION 's3://chicago-crime-data-kinjal/raw/'
TBLPROPERTIES (
  'has_encrypted_data'='false',
  'skip.header.line.count'='1'
);

'''

## 1. Upload Dataset to S3

In [ ]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'chicago-crime-data-kinjal'
file_path = 'Crimes_-_2001_to_Present.csv'
s3.upload_file(file_path, bucket_name, 'raw/Crimes_-_2001_to_Present.csv')
print("Upload complete.")

## 2. Create Athena Table

In [ ]:
from pyathena import connect

conn = connect(s3_staging_dir='s3://chicago-crime-data-kinjal/athena_output/',
               region_name='us-east-1')

create_table_query = """CREATE EXTERNAL TABLE IF NOT EXISTS chicago_crime (
    id STRING,
    case_number STRING,
    date STRING,
    primary_type STRING,
    location_description STRING,
    arrest BOOLEAN,
    domestic BOOLEAN,
    beat INT,
    district INT,
    ward INT,
    community_area INT,
    year INT,
    latitude DOUBLE,
    longitude DOUBLE
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
LOCATION 's3://chicago-crime-data-kinjal/raw/'
TBLPROPERTIES (
  'has_encrypted_data'='false',
  'skip.header.line.count'='1'
);"""

cursor = conn.cursor()
cursor.execute(create_table_query)
print("Athena table created.")

## 3. Query Athena Table

In [ ]:
query = "SELECT primary_type, COUNT(*) as cnt FROM chicago_crime GROUP BY primary_type ORDER BY cnt DESC LIMIT 10"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)